In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer , execute

import math

def get_max(num_bin1,num_bin2,sign):
    simulator = Aer.get_backend("qasm_simulator")
    '''
    This function will compare the binary represenatation of our 2 numbers using a quantum circuit
    This circuit is inspired from the paper "Quantum bit string comparator: circuits and applications"
    by "David Sena Oliveira and Rubens Viana Ramos" pg 19 where we implememt a full subtractor circuit
    and analyse the carryout qubits and the difference of (Number2 - Number1) which is stored bitwise in the 
    qubits represented by Number2. We measure the difference and carryout qubits. We then traverse
    through the difference bits to find the first instance of bit '1' . We then find the bit value of the
    corresponding carryout bit and if it is '1' then Number 1 is larger than Number 2 and if '0' then 
    Number 2 is larger than Number 1
    '''
    # We first need to find qubits required for the 2 binary numbers
    # If the length of the binaries are not same then create qubits that has max length among the 2
    if(len(num_bin1) > len(num_bin2)):
        N = len(num_bin1)
    else:
        N = len(num_bin2)

    # Qubits from register 1 represent binary number num_bin1
    qreg1 = QuantumRegister(N,'num1')
   
    # Qubits from register 2 represent binary number num_bin2
    qreg2 = QuantumRegister(N,'num2')

    # A Carry register that stores the value of carry_in from the iteration
    car_reg_prev = QuantumRegister(N,'carry_prev')
    
    # A Carry register that stores the value of carry_out from the iteration
    car_reg_current = QuantumRegister(N,'carry_current')

    # Classical registers to measure the difference and carry_outs
    cl_reg1 = ClassicalRegister(N,'classcialRegister1')
    cl_reg2 = ClassicalRegister(N,'classcialRegister2')
    
    # Creation of the QUANTUM CIRCUIT
    qc = QuantumCircuit(car_reg_prev,qreg1,qreg2,car_reg_current,cl_reg1,cl_reg2)

    # Part 1 is the encoding of the binary number into qubits
    for i in range(len(num_bin1)):
        if(num_bin1[i] == '1'):
            qc.x(qreg1[len(num_bin1) - i - 1])
    for j in range(len(num_bin2)):
        if(num_bin2[j] == '1'):
            qc.x(qreg2[len(num_bin2) - j - 1]) 

    qc.barrier()
    
    # Apply the gates to the corresponding qubits
    # NOTE : The MSB from the binary number is represented by the last qubit(bottom most) in its register
    #        and the LSB is represented by the first qubit(topmost) in its register
    for i in range(N):
        qc.cnot(car_reg_prev[N-i-1],qreg2[N-i-1])
        qc.ccx(car_reg_prev[N-i-1],qreg2[N-i-1],car_reg_current[N-i-1])
        qc.cnot(qreg1[N-i-1],qreg2[N-i-1])
        qc.ccx(qreg1[N-i-1],qreg2[N-i-1],car_reg_current[N-i-1])
        #qc.ccx(car_reg_prev[N-i-1],car_reg_current[N-i-1],anc_reg[0])
        if(N-i-2 == -1):
            break
        else:
            qc.cnot(car_reg_current[N-i-1],car_reg_prev[N-i-2])
        qc.barrier()
    qc.barrier()
    
    # Measure the qubits
    for j in range(N):
        qc.measure(car_reg_current[N-j-1],cl_reg1[N-j-1])

    for k in range(N):
        qc.measure(qreg2[N-k-1],cl_reg2[N-k-1])
    
    # Execute the circuit and get the result
    job = execute(qc, backend=simulator, shots=2**15)
    result = job.result()
    counts = result.get_counts()
    carry = str(*counts)
    #print(carry)
    
    """
    From the Results we get a dictionary which contains the key as the measured qubits and 
    the number of hits of that key as its value. This key contains the difference and the carryout value
    seperated by a blank space. The below checks for the first occurence of '1' in the difference 
    and the checks the corresponding bit from the carryout. 
    """
    signal = 0 
    if (sign == '+'):
        for m in range(int(math.floor(len(carry)/2))):
            if (carry[m] == '1'):
                if (carry[m+int((math.ceil(len(carry)/2)))] == '1'): 
                    #print("The carryout is 1, Number 1 is large",num_bin1)
                    signal = 1
                    break
                elif(carry[m+int((math.ceil(len(carry)/2)))] == '0'):
                    #print("The carryout is 0, Number 2 is large",num_bin2)
                    signal = -1
                    break
        if (signal == 1):
            return num_bin1
        elif (signal == -1):
            return num_bin2
    if (sign == '-'):
        for m in range(int(math.floor(len(carry)/2))):
            if (carry[m] == '1'):
                if (carry[m+int((math.ceil(len(carry)/2)))] == '1'): 
                    #print("The carryout is 1, Number 1 is large",num_bin1)
                    signal = -1
                    break
                elif(carry[m+int((math.ceil(len(carry)/2)))] == '0'):
                    #print("The carryout is 0 Number 2 is large",num_bin2)
                    signal = 1
                    break
        if (signal == 1):
            return num_bin1
        elif (signal == -1):
            return num_bin2

def find_the_largest_number(number_1,number_2):
    # Check if both the numbers are equal
    if (number_1 == number_2):
        return number_1
    '''
    First convert the numbers in their respective binary form
    Also replace 0b with + for non-negative inetgers
    and - for negative inetgers
    '''
    # For the first number
    if(number_1 >= 0):
        num1 = bin(number_1).replace("0b","+")
    else:
        num1 = bin(number_1).replace("-0b","-")
    # For the second number
    if(number_2 >= 0): 
        num2 = bin(number_2).replace("0b","+")
    else:
        num2 = bin(number_2).replace("-0b","-")
    #print(num1)
    #print(num2)
    '''
    When the sign of integers is not same we determine the largest among the two by checking their sign from the 
    first index of the String
    '''
    if(num1[0] != num2[0]):
        if(num1[0] == "+"):
            return number_1
        else:
            return number_2
    
    if (num1[0] == num2[0]):
        '''
        The call to get_max() is for returing the max value among the 2 integers without considering their sign
        beacuse we are calling the function only when the signs of both the integers are same.
        '''
        max_num = get_max(num1[1:],num2[1:],num1[0])
        if (num1[0] == "-"):
            largest = -1*int(max_num, 2)
        else:
            largest = int(max_num, 2)
    return largest
       
# Input for the function find_the_largest_number
A = find_the_largest_number(11,10)
print("The largest among the 2 numbers is ",A)


The largest among the 2 numbers is  11
